#### 1. Business Understanding

JCPenney is a North American department store that was founded in the year 1902 at Kemmerer Wyoming by James Cash Penney. It's headquaters is at Plano in Texas where it is registered as J.C. Penney Corporation, Inc. to trade in goods and services. In goods, JCPenney is a merchant of clothing, foot wears for men, children, women (including plus size), home products such as beddings, bath, toiletories, kitchen wares and windows accessories. Other accessories in the collection of their products include handbags, jewelries and beuaty products. In addition, they are retailers of branded product such as Nike, Levi, Stanford men's tailored clothing etc. They offer services that include styling salon, optical centers, custom decorating etc. To reach its numerous customer base, the organisation has approximately 647 physical stores outlets in America and Puetorico and maintains an appreciable online presence using its e-commerce platform (jcpenney.com, not currently available in the UK) through which it is able to market goods and services. On the average, it is reported that it reached an of approximate of 26 million online view and a staggering 65 million per month during peak holiday season. Owing to this huge online presence and face to face physical contact with customers and clienteles especially during shopping and provision of services such as salon styling, the organisation is therefore required to maintain a business success strategy that entails  customer satisfaction through excellent services, feedback with a view to mitigate intense negative feedback and churn risk. In addition, to maintain a healthy business to cutomer interaction, it is important that the database system void of data corruption to ensure that accurate customer data are used for personalised customer to business interaction, reliable, secure and accurate decision making.
Sequel to the above business understanding, the <b>objective</b> of this research is defined as follows:

1. To identify the spread of the intensity of customer sentiment across United State and Territory
2. To identify weeknesses in the collection and storage of data with a view to mititgate ineffective customer communication and combat the risk of cyber security if neccessary.

**Data to Use:**
1. all columns of reviews.csv data
2. Price and Av_Score columns from products.csv data
3. all columns from jcpenney_reviewers.json
4. all columns of jcpenney_reviewers.json
5. users.csv excluded due to similarities with jcpenney_reviewers.json

#### a. Abstract : How I answered the above questions to create the project

The strategies implemented is to go from data to actionable business solutions so as to align data initiatives with core business objectives
1. identified customers birthday - no increase in the numbers of birthdays celebrated over the period under review
2. identify duplicated user names
3. identified duplicated empty data structures including strings and list in the databases
4. vader model to gerate polarity scores
5. use kmeans algorithm to fine tune model because the dataset is unlabled
6. use higging face to generate negative and or positive sentiments
7. The use of linear regression was considered as means to establish relationship between average customer score and price of the product. The process was abandoned because there was no dependable correlation between the columns
8. result dataframe was created from the model and the outcome was used to visualize the sentiments of customers across the unite



#### b. Importing libraries and loading data

In [8]:
##### import libraries for numerical calculations, data and datetime processing
import numpy as np
import pandas as pd
import datetime as dt

# imported for visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import folium

#from folium.plugins import FastMarkerCluster
from folium.plugins import MarkerCluster
from folium.plugins import HeatMap
import branca.element as be
import branca.colormap as cm

# for processing text data, build model that generates vader score for intensity of customer ratings
import re 
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer # vader score processing
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from tqdm.notebook import tqdm

# kmeans is used to cross validate vader output and fine tune the vader score
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

# to build huggy face ai model and make comparison with other librariew
from transformers import pipeline

import warnings
warnings.filterwarnings("ignore")

In [13]:
csv_reviews_data.columns

Index(['Uniq_id', 'Username', 'Score', 'Review'], dtype='object')

In [14]:
csv_products_data.columns

Index(['Uniq_id', 'SKU', 'Name', 'Description', 'Price', 'Av_Score'], dtype='object')

In [15]:
csv_user_data.columns

Index(['Username', 'DOB', 'State'], dtype='object')

In [16]:
json_reviewers_data.columns

Index(['Username', 'DOB', 'State', 'Reviewed'], dtype='object')

In [ ]:
json_products_data.columns

In [9]:
#### Load Dataframe
csv_reviews_data = pd.read_csv('data/reviews.csv')
csv_products_data = pd.read_csv('data/products.csv')
csv_user_data = pd.read_csv('data/users.csv')

# The 'lines=True' parameter is used here for json's file to mitigate ValueError: Trailing data
json_reviewers_data = pd.read_json('data/jcpenney_reviewers.json', lines = True)
json_products_data = pd.read_json('data/jcpenney_products.json', lines=True)

In [12]:
states_lat_long_df=pd.read_csv('data/states_data.csv')#Added: Latitude and Longitude data from another source

#### 2. Data understanding and preparation - 
Explore the data and show you understand its structure and relations, with the aid of appropriate visualisation techniques. Assess the data quality, which insights you would be able to answer from it, and what preparation the data would require. Add new data from another source if required to bring new insights to the data you already have.

####  3. **Data modeling (optional)** - Would modeling be required for the insights you have considered? Use appropriate techniques, if so.
**(i). Linear Regression model to show relationship between scores and price**